# MCP Server & Client Implementation Using Fastmcp 

---
## Architecture

LLM Host (MCP Client)  <—— JSON-RPC ——>  fastmcp Server  <——> CSV Dataset


We'll:
1. Launch a server exposing two tools (`summarize`, `query`).
2. Use an MCP Client to discover and call those tools.
3. Simulate a host (agent) deciding which tool to call.
---
## Start the MCP Server

In a separate terminal, run:
```bash
python mcp_server_fastmcp.py
```
This starts an MCP-compliant server exposing `summarize` and `query` tools.

---


### 1. Install dependencies

In [1]:

%pip install fastmcp pandas langchain openai uvicorn --upgrade --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.2/603.2 kB 1.4 MB/s  0:00:00? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.8 MB/s  0:00:00m 1.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.3 MB/s  0:00:001.3 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.2/947.2 kB 1.5 MB/s  0:00:001.5 MB/s eta 0:00:01
  Attempting uninstall: openai━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━ 21/33 [opentelemetry-api]]
    Found existing installation: openai 2.6.1m╺━━━━━━━━━━━━━━ 21/33 [opentelemetry-api]
    Uninstalling openai-2.6.1:━━━━━━━━━━━━╺━━━━━━━━━━━━━━ 21/33 [opentelemetry-api]
      Successfully uninstalled openai-2.6.1╺━━━━━━━━━━━━━━ 21/33 [opentelemetry-api]
  Attempting uninstall: mcp━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 22/33 [openai]
    Found existing installation: mcp 1.16.0╸━━━━━━━━━━━━━ 22/33 [openai]
    Uninstalling mcp-1.16.0:━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 22/33 [openai]
      Successfully uninstalled mcp-1.16.0━━━━━━━━━╺━━

## Connect via MCP Client

Once the server is running, connect using the MCP client to list tools and invoke them.

In [2]:
from fastmcp import Client
from fastmcp.client import PythonStdioTransport
import asyncio

async def main():
    # Create a client using the PythonStdioTransport
    # Create a loop to run the client

    transport = PythonStdioTransport("mcp_server_fastmcp.py")
    # client = Client(transport)
    async with Client(transport) as client:
        tools = await client.list_tools()
        print("Tools:", [t.name for t in tools])

await main()


Tools: ['summarize', 'query']


## Simulate a Simple Host Agent
Let's simulate a rule-based 'AI agent' that decides whether to use `summarize` or `query` based on user text.

In [ ]:
def decide_tool(text: str):
    text = text.lower()
    if "summarize" in text or "overview" in text:
        return "summarize", {}
    if "west" in text:
        return "query", {"expr": "region == 'West' and sales > 1500"}
    return "summarize", {}

async def run_agent(user_input, client):
    tool, params = decide_tool(user_input) #"summarize", {}
    #tool = "summarize"
    #params = {}
    print(f"Agent decided to use '{tool}'")

    # API for fastmcp 2.12.5
    result = await client.call_tool(tool, params)
    # result = await client.call_tool("query", params)

    print("Result:", result, "\n")



In [ ]:
# create a connection to your MCP server
transport = PythonStdioTransport("mcp_server_fastmcp.py")

async with Client(transport) as client:
    await run_agent("Give me a summary of the dataset", client)



Agent decided to use 'summarize'
Result: CallToolResult(content=[TextContent(type='text', text='{"rows":10,"columns":["order_id","region","sales","quarter","rep"],"numeric_stats":{"order_id":{"count":10.0,"unique":"","top":"","freq":"","mean":105.5,"std":3.0276503540974917,"min":101.0,"25%":103.25,"50%":105.5,"75%":107.75,"max":110.0},"region":{"count":10,"unique":4,"top":"West","freq":4,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""},"sales":{"count":10.0,"unique":"","top":"","freq":"","mean":1235.0,"std":536.4751210965477,"min":600.0,"25%":825.0,"50%":1175.0,"75%":1450.0,"max":2200.0},"quarter":{"count":10,"unique":4,"top":"Q1","freq":3,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""},"rep":{"count":10,"unique":10,"top":"Alex","freq":1,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""}}}', annotations=None, meta=None)], structured_content={'rows': 10, 'columns': ['order_id', 'region', 'sales', 'quarter', 'rep'], 'numeric_stats': {'order_

In [ ]:
# create a connection to your MCP server
transport = PythonStdioTransport("mcp_server_fastmcp.py")

async with Client(transport) as client:
    await run_agent("Give me a summary of the dataset", client)
    await run_agent("Show West region sales > 1500", client)

---
##  Architecture Recap
```
+--------------------+      JSON-RPC (MCP)      +--------------------+
|  Host / AI Agent   |  <-------------------->  | fastmcp MCP Server |
| (LLM or Router)    |                        | (summarize, query)  |
+--------------------+                         +--------------------+
```
---